##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorBoard の移行: TensorFlow の可視化ツールキット

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/tensorboard"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/tensorboard.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/tensorboard.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/migrate/tensorboard.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>


[TensorBoard](https://www.tensorflow.org/tensorboard) は、TensorFlow で測定と可視化を提供するための組み込みツールです。精度や損失などの一般的な機械学習実験指標は、TensorBoard で追跡および可視化できます。TensorBoard は TensorFlow 1 および 2 コードと互換性があります。

TensorFlow 1 では、`tf.estimator.Estimator` はデフォルトで TensorBoard の要約を保存します。TensorFlow 2 では、`tf.keras.callbacks.TensorBoard` <a href="https://keras.io/api/callbacks/" class="external">callback</a> を使用して要約を保存できます。

このガイドでは、最初に TensorFlow 1 で Estimator を使用して TensorBoard を使用する方法を示し、次に TensorFlow 2 で同様のプロセスを実行する方法を示します。

### セットアップ

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import tempfile
import numpy as np
import datetime
%load_ext tensorboard

In [ ]:
mnist = tf.keras.datasets.mnist # The MNIST dataset.

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### TensorFlow 1: tf.estimator を使用した TensorBoard

この TensorFlow 1 の例では、`tf.estimator.DNNClassifier` をインスタンス化し、それを MNIST データセットでトレーニングおよび評価し、TensorBoard を使用して指標を表示します。

In [ ]:
%reload_ext tensorboard

feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

config = tf1.estimator.RunConfig(save_summary_steps=1,
                                 save_checkpoints_steps=1)

path = tempfile.mkdtemp()

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.1,
    model_dir=path,
    config = config
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

In [ ]:
%tensorboard --logdir {classifier.model_dir}

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_TF1.png"/> -->

### TensorFlow 2: Keras コールバックと Model.fit を使用した TensorBoard

この TensorFlow 2 の例では、`tf.keras.callbacks.TensorBoard` コールバックでログを作成して保存し、モデルをトレーニングします。コールバックは、エポックごとの精度と損失を追跡し、`callbacks` リストの `Model.fit` に渡されます。

In [ ]:
%reload_ext tensorboard

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              steps_per_execution=10)

log_dir = tempfile.mkdtemp()
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=log_dir,
  histogram_freq=1) # Enable histogram computation with each epoch.

model.fit(x=x_train,
          y=y_train,
          epochs=10,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir {tensorboard_callback.log_dir}

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_TF2.png"/> -->


# 次のステップ

- TensorBoard の詳細については、[スタートガイド](https://www.tensorflow.org/tensorboard/get_started)を参照してください。
- 下位レベルの API については、[tf.summary から TensorFlow 2 への移行](https://www.tensorflow.org/tensorboard/migrate)ガイドを参照してください。